In [1]:
!nvidia-smi

Fri Apr 16 14:43:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   36C    P0    26W / 250W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer, ElectraTokenizer
from transformers import XLMRobertaForSequenceClassification, XLMRobertaConfig, XLMRobertaTokenizer


In [1]:
MODEL_NAME1 = "bert-base-multilingual-cased"

    # electra
MODEL_NAME2 = "monologg/koelectra-base-v3-discriminator"
MODEL_NAME3 = "monologg/koelectra-small-v3-discriminator"
MODEL_NAME4 = "xlm-roberta-large"


In [5]:
model3 = XLMRobertaTokenizer.from_pretrained(MODEL_NAME4)

In [5]:
print(f'size of Bert tokenizer: {model1.vocab_size}')
print(f'size of Electra tokenizer: {model2.vocab_size}')

size of Bert tokenizer: 119547
size of Electra tokenizer: 35000


In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("/opt/ml/input/data/train/train.tsv", delimiter='\t', header=None)
df2 = pd.read_csv("/opt/ml/input/data/test/test.tsv", delimiter='\t', header=None)

In [21]:
df.head()

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님


In [9]:
temp = df.loc[0, 1]
temp_idx1 = df.loc[0, 3]
temp_idx2 = df.loc[0, 4]
temp_idx3 = df.loc[0, 6]
temp = list(temp)
# temp.insert(temp_idx1, '[NER1]')
# temp.insert(temp_idx3, '[NER2]')

In [10]:
temp = ''.join(temp)

In [11]:
temp

'영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)와 지프(Jeep)가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다.'

In [12]:
temp[temp_idx1]

'랜'

In [13]:
temp[temp_idx2]

'버'

In [14]:
temp_txt = df.loc[:10, 2] +'[sep]' +df.loc[:10, 5]

In [15]:
entity = list(temp_txt)

In [16]:
sentence = list(df.loc[:10, 1])

In [17]:
entity

['랜드로버[sep]자동차',
 '민주당[sep]27석',
 '유럽 축구 연맹[sep]UEFA',
 '강수일[sep]공격수',
 '람캄행[sep]퍼쿤 씨 인트라팃',
 'OIE[sep]전문가',
 '기아자동차[sep]K5',
 '나고야 그램퍼스[sep]토요타 자동차',
 '조선대학교[sep]조영주',
 '삼성SDS[sep]10%',
 '심상정[sep]정의당']

In [18]:
sentence

['영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)와 지프(Jeep)가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다.',
 '선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석, 비례대표 30석)을 획득하는 데 그쳤다.',
 '유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 2017년 대회부터 UEFA U-21 축구 선수권 대회 참가국을 8개국에서 12개국으로 확대하기로 결정했다.',
 '용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르비아 출신 용병 미드필더 오그넨 코로만의 부상 등이 부진의 원인으로 지적되던 가운데 인천은 시즌 마지막 4경기에서 3승 1패를 거두며 막판 승점 쌓기에 성공, 정규리그 순위 5위로 플레이오프에 진출하는 데에 성공했다.',
 '람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 부인 사이의 셋째 아들로 태어났다.',
 '이와 관련해 AFCD는 대학 소속 전문가와 세계동물보건기구(OIE)가 사람과 동물 사이에서 코로나19가 전파될 가능성이 크다고 밝혔다고 전했다.',
 '신형 K5 사전계약 돌풍에 대해 기아자동차 관계자는 "한눈에 각인될 정도의 강렬한 디자인의 영향이 컸다"고 밝혔다.',
 '2008년 이전 팀명칭은 나고야 그램퍼스 에이트였고 에이트는 나고야 시 표장에 있는 숫자(八)와 모기업 토요타 자동차의 브랜드를 상징하였으나, 2008년부터 에이트를 제외하고 팀 명칭을 나고야 그램퍼스로 호칭하고 있다.',
 'SW융합교육원 조영주 담당교수는 “4차 산업혁명에 따라 SW역량은 전문가뿐만 아니라 일반시민에게도 꼭 필요한 기술이 되고 있다”면서 “호남권 최초의 소프트웨어 중심대학인 조선대학교는 이번 프로젝트를 통해 SW교육을 접하기 힘든 환경에 있었던 분들이 SW기술에 한층 가까워지게 되어 보람을 느꼈다”고 진행 소감을 전했다.',
 '김동양 NH

In [19]:
tokenized = model1(
      entity,
      sentence,
      return_tensors="pt",
      padding=True,
      truncation=True,
      max_length=100,
      add_special_tokens=True,)

## BERT

In [20]:
temp_txt = sentence + entity

In [36]:
for txt in temp_txt:    
    tokenized_text1 = model1.tokenize(txtt)
    print(tokenized_text1)

['영국', '##에서', '사', '##용', '##되는', '스', '##포츠', '유', '##틸', '##리', '##티', '자', '##동차', '##의', '브', '##랜드', '##로는', '랜', '##드', '##로', '##버', '(', 'Land', 'Rover', ')', '와', '지', '##프', '(', 'Je', '##ep', ')', '가', '있으며', ',', '이', '브', '##랜드', '##들은', '자', '##동차', '##의', '종', '##류', '##를', '일', '##컫', '##는', '말', '##로', '사', '##용', '##되', '##기도', '한다', '.']
['선', '##거', '##에서', '민', '##주', '##당', '##은', '해', '##산', '전', '의', '##석', '##인', '230', '##석', '##에', '한', '##참', '못', '미', '##치는', '57', '##석', '(', '지역', '##구', '27', '##석', ',', '비', '##례', '##대', '##표', '30', '##석', ')', '을', '획', '##득', '##하는', '데', '그', '##쳤다', '.']
['유럽', '축구', '연', '##맹', '(', 'UEFA', ')', '집', '##행', '##위원회', '##는', '2014년', '1월', '24일', '##에', '열린', '회', '##의를', '통해', '2017년', '대회', '##부터', 'UEFA', 'U', '-', '21', '축구', '선수', '##권', '대회', '참', '##가', '##국', '##을', '8', '##개', '##국', '##에서', '12', '##개', '##국', '##으로', '확', '##대', '##하기', '##로', '결', '##정', '##했다', '.']
['용', '##병', '공', '##격', '##수', '[U

In [38]:
for txt in temp_txt:    
    
    tokenized_text2 = model2.tokenize(txt)
    print(len(tokenized_text2))
    print(tokenized_text2)

50
['영국', '##에', '##서', '사용', '##되', '##는', '스포츠', '유틸리티', '자동차', '##의', '브랜드', '##로', '##는', '랜드', '##로', '##버', '(', 'La', '##nd', 'R', '##over', ')', '와', '지프', '(', 'Je', '##ep', ')', '가', '있', '##으며', ',', '이', '브랜드', '##들', '##은', '자동차', '##의', '종류', '##를', '일컫', '##는', '말', '##로', '사용', '##되', '##기', '##도', '한다', '.']
36
['선거', '##에', '##서', '민주당', '##은', '해산', '전', '의석', '##인', '230', '##석', '##에', '한참', '못', '미치', '##는', '57', '##석', '(', '지역구', '27', '##석', ',', '비례', '##대표', '30', '##석', ')', '을', '획득', '##하', '##는', '데', '그쳤', '##다', '.']
52
['유럽', '축구', '연맹', '(', 'UEFA', ')', '집행', '##위원회', '##는', '2014', '##년', '1', '##월', '24', '##일', '##에', '열린', '회의', '##를', '통해', '2017', '##년', '대회', '##부터', 'UEFA', 'U', '-', '21', '축구', '선수', '##권', '대회', '참가', '##국', '##을', '8', '##개', '##국', '##에', '##서', '12', '##개', '##국', '##으로', '확대', '##하', '##기', '##로', '결정', '##했', '##다', '.']
79
['용병', '공격수', '챠', '##디', '##의', '부진', '##과', '시즌', '초', '활약', '##한', '강수', '##일', '##의', '침체',

In [22]:
train_unknown = set()

for i, rows in df.iterrows():
    e0, e1 = rows[2], rows[5]

    if model2.tokenize(e0)[0] == '[UNK]':
        print(i, e0, end=" ")
        train_unknown.add(e0)
    if model2.tokenize(e1)[0] == '[UNK]':
        print(i, e1, end=" ")
        train_unknown.add(e1)

1372 아르툠 미코얀 3765 蔚山放送 4074 됭케르크 4878 로퀜스 5448 竇武 8424 刘骅骐骥 

In [23]:
print(train_unknown)

{'아르툠 미코얀', '竇武', '蔚山放送', '됭케르크', '로퀜스', '刘骅骐骥'}


In [ ]:
additional_df1 = 